In [3]:
import pandas as pd
import pyarrow.parquet as pq

path = "/proj/rep-learning-robotics/users/x_alblo/openpi/lerobot_home/datasets/1pen/data/chunk-000/episode_000000.parquet"
table = pq.read_table(path)
print(table.column_names)
print(table["image"][0])

['image', 'depth_image', 'state', 'actions', 'timestamp', 'frame_index', 'episode_index', 'index', 'task_index']
[('bytes', b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01@\x00\x00\x00\xf0\x08\x02\x00\x00\x00\xfeO*<\x00\x00\xa5VIDATx\x9c\xed\xfdY\xaf5I\x92\x18\x88\x99\x99G\x9cs\xee\xf2\xad\xb9U\xd6\x92\xb5d\xb3\xa9*v79\x10\x9b"\xa9Qc\x06\x02(\tZ\x1e\t\x01"\xf4\x03\xc8\x17\x01\xf3\x03$\xe8\x07\x08\x10\xf82o"\x04>\x0e%\n \x04\x0e\x01\x12`\xcfpi\x8aC\xb2\x17v\xa3\xbb\xbak\xc9\xca\xac\xaa\\\xbe\xf5n\xe7\x9c\x087\xd3\x83\xb9{X\xb8{\xc4\x89s\xef\xfd\xb2j\x04\x19\x12_\xc6\x8d\xe3\xe1n\xeen\xbb\x9b\xbb\xe3\xf5?\xfe?\x00\x80\x88\x00\x80>\xa0>\x01\x03\x80\xf4^D\xd2\xaf\x00\xe0\x01E\xc4\xfb\x9d\x88x\x0e\xff2\xb3g\xf0>/\x0c\x00\x9e[\x00\xe0\xe6RD@V\x00\x00\xfd\xdb\x00\xc0\xb8cf\x81ND\x04nD\x04\xb0\x07\x00\x00\x02\x00\x90\x06\x00\x80O\x01@\xbf\xda\xaf.\x88\xe8\xac\xc7\xd3\xd3\xd3\xd3\xb6s\xce}\xff\xf9{\xff\xf4\x9f\xfe\xd3\x8f\xe5\xd7\xbc\xf7\xaf\xdd\xce9wr\xbaFD\xc2\r\x11\xed\xaeo\x00\xc0\xafa\xb

In [4]:
from datasets import load_dataset
ds = load_dataset("parquet", data_dir="/proj/rep-learning-robotics/users/x_alblo/openpi/lerobot_home/datasets/1pen/data/chunk-000", split="train")
print(ds.features)

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

{'image': Image(mode=None, decode=True, id=None), 'depth_image': Image(mode=None, decode=True, id=None), 'state': Sequence(feature=Value(dtype='float32', id=None), length=6, id=None), 'actions': Sequence(feature=Value(dtype='float32', id=None), length=7, id=None), 'timestamp': Value(dtype='float32', id=None), 'frame_index': Value(dtype='int64', id=None), 'episode_index': Value(dtype='int64', id=None), 'index': Value(dtype='int64', id=None), 'task_index': Value(dtype='int64', id=None)}


In [5]:
# Diagnostics for Parquet loading
import os
import pyarrow.parquet as pq

path = "/proj/rep-learning-robotics/users/x_alblo/openpi/lerobot_home/datasets/1pen/data/chunk-000/episode_000000.parquet"
print("File exists:", os.path.exists(path))

try:
    schema = pq.read_schema(path)
    print("Schema:")
    for f in schema:
        print(" -", f.name, ":", f.type)
except Exception as e:
    print("Failed to read schema:", repr(e))
    raise

try:
    table = pq.read_table(path)
    print("Columns:", table.column_names)
    # Safely access first image bytes
    img_col = table.column("image")
    first = img_col[0]
    try:
        val = first.as_py()  # returns bytes for binary columns
    except Exception:
        val = first
    if isinstance(val, (bytes, bytearray)):
        print("image[0]: bytes length:", len(val))
        print("image[0]: first 32 bytes:", val[:32])
    else:
        print("image[0]:", type(val), str(val)[:120])
except Exception as e:
    print("Failed to read table or access first image:", repr(e))
    raise


File exists: True
Schema:
 - image : struct<bytes: binary, path: string>
 - depth_image : struct<bytes: binary, path: string>
 - state : fixed_size_list<element: float>[6]
 - actions : fixed_size_list<element: float>[7]
 - timestamp : float
 - frame_index : int64
 - episode_index : int64
 - index : int64
 - task_index : int64
Columns: ['image', 'depth_image', 'state', 'actions', 'timestamp', 'frame_index', 'episode_index', 'index', 'task_index']
image[0]: <class 'dict'> {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01@\x00\x00\x00\xf0\x08\x02\x00\x00\x00\xfeO*<\x00\x00\xa5VIDAT


In [8]:

# Step-by-step: replicate LeRobotDataset internal loading using HF datasets
from datasets import load_dataset
import os
from pathlib import Path
from lerobot.common.datasets.utils import hf_transform_to_torch

repo_root = Path(os.environ.get("HF_LEROBOT_HOME", "/proj/rep-learning-robotics/users/x_alblo/openpi/lerobot_home/datasets")) / "1pen"
parquet_dir = str(repo_root / "data")
print("parquet_dir:", parquet_dir, "exists:", Path(parquet_dir).exists())

# Try the same call used inside LeRobotDataset
try:
    hf_ds = load_dataset("parquet", data_dir=parquet_dir, split="train")
    hf_ds.set_transform(hf_transform_to_torch)
    print("Loaded HF dataset. Num rows:", len(hf_ds))
    print("HF features:", hf_ds.features)
except Exception as e:
    print("HF load_dataset error:", repr(e))
    raise

ts = hf_ds['timestamp']
import torch
timestamps = torch.stack(ts).numpy()


parquet_dir: /proj/rep-learning-robotics/users/x_alblo/openpi/lerobot_home/datasets/1pen/data exists: True


Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loaded HF dataset. Num rows: 4710
HF features: {'image': Image(mode=None, decode=True, id=None), 'depth_image': Image(mode=None, decode=True, id=None), 'state': Sequence(feature=Value(dtype='float32', id=None), length=6, id=None), 'actions': Sequence(feature=Value(dtype='float32', id=None), length=7, id=None), 'timestamp': Value(dtype='float32', id=None), 'frame_index': Value(dtype='int64', id=None), 'episode_index': Value(dtype='int64', id=None), 'index': Value(dtype='int64', id=None), 'task_index': Value(dtype='int64', id=None)}


In [6]:
# Inspect timestamp column types and first values
import itertools

if 'hf_ds' in globals():
    ts = hf_ds['timestamp']
    # Show types of first 20 values
    sample = list(itertools.islice(iter(ts), 20))
    types = {type(x).__name__ for x in sample}
    print("timestamp sample types:", types)
    print("first 5 values:", sample[:5])
else:
    print("hf_ds not loaded in previous cell.")


timestamp sample types: {'float'}
first 5 values: [0.0, 0.10000000149011612, 0.20000000298023224, 0.30000001192092896, 0.4000000059604645]


In [7]:
# Try to stack timestamps like LeRobotDataset does
import torch

if 'hf_ds' in globals():
    ts = hf_ds['timestamp']
    try:
        stacked = torch.stack([torch.tensor(x) for x in ts])
        print("stacked shape:", stacked.shape, stacked.dtype)
    except Exception as e:
        print("torch.stack failure:", repr(e))
        # Diagnose problematic entries
        bad_idxs = []
        for i, x in enumerate(ts[:1000]):
            try:
                _ = torch.tensor(x)
            except Exception:
                bad_idxs.append((i, type(x).__name__, str(x)[:120]))
                if len(bad_idxs) >= 10:
                    break
        print("First problematic entries:", bad_idxs)
else:
    print("hf_ds not loaded in previous cell.")


stacked shape: torch.Size([4710]) torch.float32


In [ ]:
# Configure local vs HF loading for LeRobot datasets
# Option A (local): set LOCAL_LEROBOT_HOME to a directory containing a folder named REPO_NAME
#   e.g., LOCAL_LEROBOT_HOME=/data/lerobot_home and REPO_NAME="my_dataset" -> dataset at /data/lerobot_home/my_dataset
# Option B (HF Hub/cache): leave LOCAL_LEROBOT_HOME=None and set CUSTOM_REPO_ID to your hub repo id
LOCAL_LEROBOT_HOME = "/proj/rep-learning-robotics/users/x_alblo/openpi/lerobot_home/datasets"
REPO_NAME = "1pen"
CUSTOM_REPO_ID = "ignored_when_LOCAL_LEROBOT_HOME_is_set"

# Optional: if your dataset encodes the language prompt inside the LeRobot "task" field
PROMPT_FROM_TASK = False


import os
import sys
from pathlib import Path

# If using a local LeRobot home, export HF_LEROBOT_HOME so LeRobotDataset resolves locally
if LOCAL_LEROBOT_HOME:
    os.environ["HF_LEROBOT_HOME"] = str(LOCAL_LEROBOT_HOME)
    CUSTOM_REPO_ID = REPO_NAME  # refer to the folder name under HF_LEROBOT_HOME

# Ensure project root is on sys.path for imports
project_root = Path("/proj/rep-learning-robotics/users/x_alblo/openpi")
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.openpi.training import config as _config
from src.openpi.training import data_loader as _data
from src.openpi.models import pi0_config

# Build DataConfigFactory for our dataset
data_factory = _config.KTHPenDataConfig(
    repo_id=CUSTOM_REPO_ID,
    base_config=_config.DataConfig(
        prompt_from_task=PROMPT_FROM_TASK,
    ),
    extra_delta_transform=False,
)

# Minimal TrainConfig mirroring training setup, but used here for inspection
cfg = _config.TrainConfig(
    name="inspect_custom_dataset",
    exp_name="inspect_custom_dataset",
    model=pi0_config.Pi0Config(),
    data=data_factory,
    batch_size=4,
    num_workers=0,  # keep 0 for notebooks
)


# Create the unified data loader (use PyTorch framework for simplicity in notebooks)
try:
    dl = _data.create_data_loader(cfg, framework="pytorch", shuffle=False, skip_norm_stats=True)
except Exception as e:
    print("Failed to create data loader:", e)
    raise

# Fetch a single batch to inspect shapes/content
try:
    batch = next(iter(dl))
except Exception as e:
    print("Failed to fetch a batch:", e)
    raise
obs, acts = batch

# Utility to summarize arrays/structures
import numpy as np

def array_info(x):
    try:
        arr = np.array(x)
        return {"shape": arr.shape, "dtype": arr.dtype}
    except Exception:
        return type(x).__name__

print("Images keys:", list(obs.images.keys()))
for k, v in obs.images.items():
    print("image[", k, "] ->", array_info(v))

print("Proprio state:", array_info(obs.state) if getattr(obs, "state", None) is not None else None)
print("Prompt:", obs.prompt[:1] if getattr(obs, "prompt", None) is not None else None)
print("Actions:", array_info(acts))


Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Failed to create data loader: stack(): argument 'tensors' (position 1) must be tuple of Tensors, not Column


TypeError: stack(): argument 'tensors' (position 1) must be tuple of Tensors, not Column

In [ ]:
# Visualize first sample images (if available)
import matplotlib.pyplot as plt
import numpy as np

first_key = None
for k in obs.images.keys():
    first_key = k
    break

if first_key is not None:
    imgs = obs.images[first_key]
    arr = np.array(imgs)
    # Expected shape [B, T, H, W, C] or [B, H, W, C]; try to squeeze
    while arr.ndim > 4:
        arr = arr[:, 0]
    if arr.ndim == 4:
        # show first in batch
        img = arr[0]
        plt.figure(figsize=(4, 4))
        plt.title(f"{first_key} (first sample)")
        plt.imshow(img)
        plt.axis('off')
        plt.show()
else:
    print("No image keys found to visualize.")
